# SimResults To XWiki Tables - Race Reports and Championship Standings

In [19]:
import pandas as pd
import numpy as np
import re, os

### Race Report Generator

In [20]:
class RaceReport():
    
    # Race report table format strings
    qualy_row_0 = """(% border="1" style="width:554px" %)"""
    qualy_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 233px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 77px; text-align: center;" %)Time"""
    qualy_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:77px" %){}"""
    qualy_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:77px" %){}"""
    qualy_row_last = """|=(% colspan="5" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 552px;" %)[[Source>>{}]]"""
    race_row_0 = """(% border="1" style="width:747px" %)"""
    race_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 229px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)Laps|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 111px; text-align: center;" %)Time/Retired|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 53px; text-align: center;" %)Grid|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 62px; text-align: center;" %)Points"""
    race_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:44px" %){}|(% style="text-align:center; border-color:#000000; width:111px" %){}|(% style="text-align:center; border-color:#000000; width:53px" %){}|(% style="text-align:center; border-color:#000000; width:62px" %){}"""
    race_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:44px" %){}|(% style="text-align:center; width:111px" %){}|(% style="text-align:center; width:53px" %){}|(% style="text-align:center; width:62px" %){}"""
    race_row_last = """|=(% colspan="8" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 745px;" %)[[Source>>{}]]"""
    race_row_fastestlap = """Fastest lap:  [[image:{}||height="14" width="23"]] {} - {}"""
    
    
    # Read series drivers info
    def read_drivers_table(series_directory):
        drivers_table_file = "{}/drivers_table.csv".format(series_directory)
        table = pd.read_csv(drivers_table_file, dtype=object)
        table = table.set_index("ign")
        return table
    
    
    # Read series points scoring info
    def read_points_table(series_directory):
        points_table_file = "{}/points_table.csv".format(series_directory)
        table = pd.read_csv(points_table_file, dtype=object)
        table = table.astype(int)
        table = table.set_index("pos")
        return table
    
    
    # Read series track info (used only for championship table)
    def read_tracks_table(series_directory):
        tracks_table_file = "{}/tracks_table.csv".format(series_directory)
        table = pd.read_csv(tracks_table_file, dtype=object)
        table["csv_manual_adjustment"] = table["csv_manual_adjustment"].astype(int)
        table = table.set_index("directory")
        return table
    
    
    # Constructor, optionally pass in already-parsed drivers and points table info, debug to print intermediate dataframes
    def __init__(self, table_names, series_directory, race_directory, drivers_table=None, points_table=None, csv_manual_adjustment=0, output_file_name="wiki_tables.txt", debug=False):
        
        self.table_names = table_names
        self.debug = debug
        
        if self.debug: print("Creating race report:", series_directory, race_directory)
        
        self.drivers_table = drivers_table if drivers_table is not None else RaceReport.read_drivers_table(series_directory)
        self.points_table = points_table if points_table is not None else RaceReport.read_points_table(series_directory)

        race_directory_path = "{}/{}".format(series_directory, race_directory)
        assert os.path.isdir(race_directory_path), "Race directory path does not exist: {}".format(race_directory_path)
        
        self.output_file = "{}/{}".format(race_directory_path, output_file_name)
        
        csv_files = [file for file in os.listdir(race_directory_path) if file.endswith(".csv")]
        if len(csv_files) < 1:
            raise FileNotFoundError()
        simresults_file_name = [file for file in os.listdir(race_directory_path) if file.endswith(".csv")][0]
        simresults_code = os.path.splitext(simresults_file_name)[0]
        self.results_file = "{}/{}.csv".format(race_directory_path, simresults_code)
        self.simresults_url = "https://simresults.net/{}".format(simresults_code)
        
        self.csv_manual_adjustment = csv_manual_adjustment
        self.tables = self.__read_results_tables()
        self.__clean_results_tables()
    
    
    # Return race result pandas dataframes in dict keyed by table_names
    def __read_results_tables(self):
        
        rows = {name: [0,0] for name in self.table_names}
        with open(self.results_file) as fp:
            current_table = ""
            for i, row in enumerate(fp):
                if current_table:
                    if row=="\n":
                        rows[current_table][1] = i
                        if self.debug: print("ending line {}".format(rows[current_table][1]))
                        current_table = ""
                    else:
                        if self.debug: print(row, end="")
                else:
                    for name in self.table_names:
                        if row.startswith(name):
                            current_table = name
                            rows[current_table][0] = i+2
                            if self.debug: print("found table '{}' starting line {} ".format(current_table, rows[current_table][0]))
        
        tables = {}
        for name in self.table_names:
            table_skiprows = rows[name][0] + self.csv_manual_adjustment
            table_nrows = rows[name][1] - table_skiprows - 1 + self.csv_manual_adjustment
            table_df = pd.read_csv(self.results_file, skiprows=table_skiprows, nrows=table_nrows, index_col=False, skipinitialspace=True, quotechar='"', dtype=object)
            if self.debug: display(table_df)
            tables[name] = table_df
        
        return tables
    
    
    # Clean race result dataframes, cast numerical columns to integers, join points information and starting positions to driver rows
    def __clean_results_tables(self):
        
        # Merge starting position info to grid column of a race table from either quali session or previous race, optionally adding quali points column
        def merge_qualy_info(race_table, qualy_table, add_qualy_points=False):
            
            qualy_table = qualy_table.reset_index()
            race_table = race_table.reset_index()
            
            if add_qualy_points:
                qualy_table = qualy_table[["Pos", "Driver", "Points"]]
                qualy_table = qualy_table.rename(columns={"Pos":"Grid", "Points":"Qualify Points"})
            else:
                qualy_table = qualy_table[["Pos", "Driver"]]
                qualy_table = qualy_table.rename(columns={"Pos":"Grid"})

            race_table = race_table.merge(qualy_table, how='left', on="Driver")

            race_table["Grid"] = race_table["Grid"].fillna(-1)
            # Pandas issue where NaN values cause ints to become floats?
            race_table["Grid"] = race_table["Grid"].astype(int)
            race_table["Pos"] = race_table["Pos"].astype(int)
            race_table["Laps"] = race_table["Laps"].astype(int)

            if add_qualy_points:
                race_table["Qualify Points"] = race_table["Qualify Points"].fillna(0)
                race_table["Qualify Points"] = race_table["Qualify Points"].astype(int)
                
            race_table = race_table.set_index("Pos")

            return race_table
        
        # TODO remove quote stripping since quotechar fixes this for us
        # Strip quotes and whitespace from strings, cast position and laps to integers, drop unnamed columns and rows for non-participants, convert fastest laps to datetimes
        for name, table_df in self.tables.items():

            table_df = table_df.apply(lambda s: s.str.strip(' \'"'), axis=1)
            table_df = table_df.rename(columns=lambda c: c.strip(' \'"'))
            table_df = table_df.apply(lambda s: s.str.replace('(.*\d\d*\.\d{3})0$', r'\1'), axis=1)

            table_df["Pos"] = table_df["Pos"].astype(int)
            table_df = table_df.set_index("Pos")
            
            table_df["Laps"] = table_df["Laps"].astype(int)

            table_df = table_df[table_df["Laps"] > 0]

            table_df = table_df[table_df.columns.drop(list(table_df.filter(regex='Unnamed*')))]
            table_df = table_df.dropna(how='all', axis='columns')
            
            table_df["Best lap time"] = pd.to_datetime(table_df["Best lap"], format="%M:%S.%f", errors='coerce')

            self.tables[name] = table_df
            
            
        # Merge points column to both quali and race sessions, for race sessions: get attatched qualifying or previous race to get starting positions and show (probable) DNFs
        for name, table_df in self.tables.items():

            if name.startswith("Qualify"):

                points_column = pd.Series(np.zeros(len(table_df))).add(self.points_table["qualy_points"].astype(int), fill_value=0)
                table_df["Points"] = points_column.astype(int)

            if name.startswith("Race"):

                table_df["Time/Retired"] = table_df["Time/Retired"].str.replace('^\s*$', "")
                table_df["Consistency"] = table_df["Consistency"].str.replace('^-$', "")

                points_column = pd.Series(np.zeros(len(table_df)+1)).add(self.points_table["points"].astype(int), fill_value=0)
                table_df["Points"] = points_column.astype(int)

                session_num = int(re.findall('(?:Qualify|Race)\s(\d+)\sresult|$', name)[0])
                qualy_table_name = "Qualify result" if session_num==1 and "Qualify result" in self.tables.keys() else "Qualify {} result".format(session_num)
                previous_race_table_name = "Race {} result".format(session_num - 1)
                if qualy_table_name in self.tables:
                    table_df = merge_qualy_info(table_df, self.tables[qualy_table_name], add_qualy_points=True)
                elif previous_race_table_name in self.tables:
                    table_df = merge_qualy_info(table_df, self.tables[previous_race_table_name])
                else:
                    table_df["Grid"] = ""
                    
                table_df["DNF"] = table_df["Laps"] < table_df["Laps"].max()//2
                
            # Validate drivers are in the driver table
            for driver in table_df["Driver"]:
                assert driver in self.drivers_table.index, "{} not found in drivers table".format(driver)

            if self.debug: display(table_df)
            self.tables[name] = table_df
        

    # Generate quali table markdown from quali session dataframe, result is a list of lines
    def __generate_qualy_table_strings(self, table_name):

        table_df = self.tables[table_name]

        lines_buffer = [table_name, self.qualy_row_0, self.qualy_row_1]

        for i, df_row in table_df.iterrows():

            driver = df_row["Driver"]

            driver_info = self.drivers_table.loc[driver]

            position = i
            number = driver_info["number"]
            flag = driver_info["flag"]
            name = driver_info["name"]
            team = driver_info["team"]
            time = df_row["Best lap"]
            if position==1:
                time = "**{}**".format(time)

            line = self.qualy_row_2.format(position, number, flag, name, team, time) if i==0 else self.qualy_row.format(position, number, flag, name, team, time)
            lines_buffer.append(line)

        lines_buffer.append(self.qualy_row_last.format(self.simresults_url))

        if self.debug: print("made {} rows for table: {}".format(len(lines_buffer), table_name))
        return lines_buffer
    
    
    def __generate_race_table_strings(self, table_name):
    
        table_df = self.tables[table_name]

        lines_buffer = [table_name, self.race_row_0, self.race_row_1]

        for i, df_row in table_df.iterrows():

            driver = df_row["Driver"]

            driver_info = self.drivers_table.loc[driver]

            position = i
            number = driver_info["number"]
            flag = driver_info["flag"]
            name = driver_info["name"]
            team = driver_info["team"]
            laps = df_row["Laps"]
            points = df_row["Points"]
            timeorretired = df_row["Time/Retired"] if df_row["Time/Retired"] else "DNF"
            grid = df_row["Grid"] if df_row["Grid"]>0 else "DNQ"

            if "Qualify Points" in table_df.columns:
                points = str(df_row["Points"] + df_row["Qualify Points"]) + ("^^{}^^".format(grid) if df_row["Qualify Points"]>0 else "")

            line = self.race_row_2.format(position, number, flag, name, team, laps, timeorretired, grid, points) if i==0 else self.race_row.format(position, number, flag, name, team, laps, timeorretired, grid, points)
            lines_buffer.append(line)

        lines_buffer.append(self.race_row_last.format(self.simresults_url))
        
        fastest_driver = table_df.loc[table_df["Best lap time"]==table_df["Best lap time"].min()].iloc[0]["Driver"]
        fastest_driver_flag = self.drivers_table.loc[fastest_driver, "flag"]
        
        fastest_time = table_df.loc[table_df["Driver"]==fastest_driver]["Best lap"].item()
        lines_buffer.append(self.race_row_fastestlap.format(fastest_driver_flag, fastest_driver, fastest_time))

        if self.debug: print("made {} rows for table: {}".format(len(lines_buffer), table_name))
        return lines_buffer
    
    
    # Generate table markdown for all table_names, result is map from table names to table markdown string
    def generate_tables_strings(self):
        tables_strings = {}

        for name in self.table_names:
            lines_buffer = ["error!"]
            if name.startswith("Qualify"):
                lines_buffer = self.__generate_qualy_table_strings(name)
            elif name.startswith("Race"):
                lines_buffer = self.__generate_race_table_strings(name)
            tables_strings[name] = "\n".join(lines_buffer) + "\n\n"
            
        return tables_strings
    
    
    # Write table markdown for all table_names, optionally provide the generated table strings
    def write_generated_tables(self, tables_strings=None):
        if not tables_strings:
            tables_strings = self.generate_tables_strings()
            
        with open(self.output_file, "w+") as fp:
            for name in self.table_names:
                fp.write(tables_strings[name])
                if self.debug: print("wrote table {} to {}".format(name, self.output_file))

    

### Read Series Info

In [21]:
series = "F1H"
series_sessions = ["Qualify result", "Race 1 result", "Race 2 result"]
rounds_to_include = 4
drop_week = False

series_drivers_table = RaceReport.read_drivers_table(series)
display(series_drivers_table)

series_points_table = RaceReport.read_points_table(series)
display(series_points_table)

series_tracks_table = RaceReport.read_tracks_table(series)
display(series_tracks_table)

series_race_sessions = [session for session in series_sessions if session.startswith("Race")]
num_total_races = len(series_tracks_table) * len(series_race_sessions)

,name,number,team,flag
ign,,,,
Electro,Electro,25,Scuderia Minardi,https://upload.wikimedia.org/wikipedia/commons...
gunciz,Gunciz,000,Independent,https://upload.wikimedia.org/wikipedia/commons...
Viperion_NZ,Viperion_NZ,05,Independent,https://upload.wikimedia.org/wikipedia/commons...
Georgin,Georgin,34,Andrea Moda Formula,https://upload.wikimedia.org/wikipedia/en/thum...
McLarenTim,McLarenTim,71,Monterey SimSport,https://upload.wikimedia.org/wikipedia/en/thum...
Salzeder,Salzeder,16,Scuderia Minardi,https://upload.wikimedia.org/wikipedia/en/thum...
NotJames,NotJames,73,Monterey SimSport,https://upload.wikimedia.org/wikipedia/en/thum...
"G, T",Gman197002,19,Independent,https://upload.wikimedia.org/wikipedia/en/thum...
Trempale,Trempale,17,Jordan Grand Prix,https://upload.wikimedia.org/wikipedia/en/thum...


,points,qualy_points
pos,,
1,40,3
2,36,2
3,32,1
4,28,0
5,25,0
6,22,0
7,19,0
8,16,0
9,13,0


,abbrev,full_name,flag,csv_manual_adjustment
directory,,,,
monza_1,MON,Monza 1966 Full Course,https://upload.wikimedia.org/wikipedia/en/thum...,0
monza_2,MON,Monza 1966 Road Course,https://upload.wikimedia.org/wikipedia/en/thum...,0
imola_3,IML,Imola 1994 GP Circuit,https://upload.wikimedia.org/wikipedia/en/thum...,-1
imola_4,IML,Imola 1994 GP Circuit,https://upload.wikimedia.org/wikipedia/en/thum...,0
imola_5,IML,Imola 1994 GP Circuit,https://upload.wikimedia.org/wikipedia/en/thum...,0
imola_6,IML,Imola 2009 GP Circuit,https://upload.wikimedia.org/wikipedia/en/thum...,0
imola_7,IML,Imola 2009 GP Circuit,https://upload.wikimedia.org/wikipedia/en/thum...,0
imola_8,IML,Imola 2009 GP Circuit,https://upload.wikimedia.org/wikipedia/en/thum...,0


### Read Race Results

In [22]:
race_reports = {}

for race, race_row in series_tracks_table.iterrows():
    
    race_path = "{}/{}".format(series, race)
    if os.path.isdir(race_path):
        try:
            race_reports[race] = RaceReport(series_sessions, series, race, drivers_table=series_drivers_table, points_table=series_points_table, csv_manual_adjustment=race_row["csv_manual_adjustment"], debug=True)
            print(race)
            [display(race_reports[race].tables[session]) for session in series_race_sessions]
        except FileNotFoundError:
            print("no csv found for", race)
    else:
        print("no directory found for", race)

Creating race report: F1H monza_1
found table 'Qualify result' starting line 15 
'======================
Pos,   Class, Team, Vehicle, Driver, Laps, Best lap, Gap
"1",   "", "", "ks_maserati_250f_12cyl", "gunciz", "3", "02:42.3430", "-",
"2",   "", "", "ks_maserati_250f_12cyl", "Electro", "3", "02:43.1290", "00:00.7860",
"3",   "", "", "ks_maserati_250f_12cyl", "Waffles", "3", "02:43.1440", "00:00.8010",
"4",   "", "", "ks_maserati_250f_12cyl", "MrGrinder", "4", "02:43.2250", "00:00.8820",
"5",   "", "", "ks_maserati_250f_12cyl", "NotJames", "3", "02:43.2780", "00:00.9350",
"6",   "", "", "ks_maserati_250f_12cyl", "McLarenTim", "3", "02:43.7430", "00:01.4000",
"7",   "", "", "ks_maserati_250f_12cyl", "Smartpipe", "3", "02:45.2710", "00:02.9280",
"8",   "", "", "ks_maserati_250f_12cyl", "Georgin", "3", "02:46.2260", "00:03.8830",
"9",   "", "", "ks_maserati_250f_12cyl", "marzen224", "2", "02:49.7140", "00:07.3710",
"10",   "", "", "ks_maserati_250f_12cyl", "Viperion_NZ", "2", "02:50.4860

,Pos,Class,Team,Vehicle,Driver,Laps,Best lap,Gap
0,1,NaN,NaN,ks_maserati_250f_12cyl,gunciz,3,02:42.3430,-
1,2,NaN,NaN,ks_maserati_250f_12cyl,Electro,3,02:43.1290,00:00.7860
2,3,NaN,NaN,ks_maserati_250f_12cyl,Waffles,3,02:43.1440,00:00.8010
3,4,NaN,NaN,ks_maserati_250f_12cyl,MrGrinder,4,02:43.2250,00:00.8820
4,5,NaN,NaN,ks_maserati_250f_12cyl,NotJames,3,02:43.2780,00:00.9350
5,6,NaN,NaN,ks_maserati_250f_12cyl,McLarenTim,3,02:43.7430,00:01.4000
6,7,NaN,NaN,ks_maserati_250f_12cyl,Smartpipe,3,02:45.2710,00:02.9280
7,8,NaN,NaN,ks_maserati_250f_12cyl,Georgin,3,02:46.2260,00:03.8830
8,9,NaN,NaN,ks_maserati_250f_12cyl,marzen224,2,02:49.7140,00:07.3710
9,10,NaN,NaN,ks_maserati_250f_12cyl,Viperion_NZ,2,02:50.4860,00:08.1430


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Unnamed: 11
0,1,NaN,NaN,ks_maserati_250f_12cyl,NotJames,8,21:58.8630,02:42.6890,99.54%,6,0,NaN
1,2,NaN,NaN,ks_maserati_250f_12cyl,Electro,8,'+00:07.6670,02:42.2130,98.84%,0,0,NaN
2,3,NaN,NaN,ks_maserati_250f_12cyl,MrGrinder,8,'+00:09.6990,02:43.1410,98.88%,0,0,NaN
3,4,NaN,NaN,ks_maserati_250f_12cyl,Waffles,8,'+00:19.5800,02:43.6790,98.21%,1,0,NaN
4,5,NaN,NaN,ks_maserati_250f_12cyl,gunciz,8,'+00:20.9890,02:42.6300,97.44%,0,0,NaN
5,6,NaN,NaN,ks_maserati_250f_12cyl,McLarenTim,8,'+00:36.5300,02:43.1370,99.53%,0,0,NaN
6,7,NaN,NaN,ks_maserati_250f_12cyl,Georgin,8,'+00:39.1340,02:45.5970,98.57%,0,0,NaN
7,8,NaN,NaN,ks_maserati_250f_12cyl,Smartpipe,8,'+00:43.5860,02:43.6530,96.89%,0,0,NaN
8,9,NaN,NaN,ks_maserati_250f_12cyl,Viperion_NZ,8,'+00:59.1350,02:44.9510,98.39%,0,0,NaN
9,10,NaN,NaN,ks_maserati_250f_12cyl,marzen224,8,'+01:50.5280,02:51.4100,95.91%,0,0,NaN


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Unnamed: 11
0,1,NaN,NaN,ks_maserati_250f_12cyl,Electro,8,22:02.5340,02:42.0340,98.96%,0,0,NaN
1,2,NaN,NaN,ks_maserati_250f_12cyl,McLarenTim,8,'+00:03.8030,02:42.3570,98.55%,0,0,NaN
2,3,NaN,NaN,ks_maserati_250f_12cyl,NotJames,8,'+00:06.9590,02:42.7800,99.28%,1,0,NaN
3,4,NaN,NaN,ks_maserati_250f_12cyl,Waffles,8,'+00:07.5190,02:43.5290,99.03%,0,0,NaN
4,5,NaN,NaN,ks_maserati_250f_12cyl,gunciz,8,'+00:27.1310,02:42.9660,98.49%,0,0,NaN
5,6,NaN,NaN,ks_maserati_250f_12cyl,Georgin,8,'+00:44.6780,02:43.9220,97.62%,0,0,NaN
6,7,NaN,NaN,ks_maserati_250f_12cyl,MrGrinder,8,'+02:35.4050,02:43.2270,97.36%,0,0,NaN
7,8,NaN,NaN,ks_maserati_250f_12cyl,Smartpipe,7,'+-00:01.7370,02:42.9470,99.48%,6,0,NaN
8,9,NaN,NaN,ks_maserati_250f_12cyl,marzen224,6,'+01:18.6650,02:47.2710,96.31%,0,0,NaN
9,10,NaN,NaN,ks_maserati_250f_12cyl,Viperion_NZ,0,DNF,-,-,0,0,NaN


,Vehicle,Driver,Laps,Best lap,Gap,Best lap time,Points
Pos,,,,,,,
1,ks_maserati_250f_12cyl,gunciz,3,02:42.343,-,1900-01-01 00:02:42.343,3
2,ks_maserati_250f_12cyl,Electro,3,02:43.129,00:00.786,1900-01-01 00:02:43.129,2
3,ks_maserati_250f_12cyl,Waffles,3,02:43.144,00:00.801,1900-01-01 00:02:43.144,1
4,ks_maserati_250f_12cyl,MrGrinder,4,02:43.225,00:00.882,1900-01-01 00:02:43.225,0
5,ks_maserati_250f_12cyl,NotJames,3,02:43.278,00:00.935,1900-01-01 00:02:43.278,0
6,ks_maserati_250f_12cyl,McLarenTim,3,02:43.743,00:01.400,1900-01-01 00:02:43.743,0
7,ks_maserati_250f_12cyl,Smartpipe,3,02:45.271,00:02.928,1900-01-01 00:02:45.271,0
8,ks_maserati_250f_12cyl,Georgin,3,02:46.226,00:03.883,1900-01-01 00:02:46.226,0
9,ks_maserati_250f_12cyl,marzen224,2,02:49.714,00:07.371,1900-01-01 00:02:49.714,0


...

,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,DNF
Pos,,,,,,,,,,,,
1,lotus_98t,NotJames,13,19:44.906,01:29.968,99.41%,11,0,1900-01-01 00:01:29.968,40,3,False
2,lotus_98t,gunciz,13,+00:05.046,01:29.492,98.50%,0,0,1900-01-01 00:01:29.492,36,4,False
3,lotus_98t,Electro,13,+00:05.641,01:28.960,98.88%,0,0,1900-01-01 00:01:28.960,32,1,False
4,lotus_98t,McLarenTim,13,+00:09.548,01:28.835,97.20%,0,0,1900-01-01 00:01:28.835,28,2,False
5,lotus_98t,MrGrinder,13,+00:28.254,01:29.647,97.63%,0,0,1900-01-01 00:01:29.647,25,7,False
6,lotus_98t,Trempale,13,+00:39.579,01:30.105,97.67%,0,0,1900-01-01 00:01:30.105,22,8,False
7,lotus_98t,Salzeder,13,+00:42.971,01:30.983,97.64%,0,0,1900-01-01 00:01:30.983,19,6,False
8,lotus_98t,marzen224,12,+01:48.597,01:34.680,96.80%,0,0,1900-01-01 00:01:34.680,16,9,False
9,lotus_98t,"G, T",12,+02:27.599,01:34.833,90.67%,0,0,1900-01-01 00:01:34.833,13,10,False


imola_4


,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,Qualify Points,DNF
Pos,,,,,,,,,,,,,
1,lotus_98t,Electro,13,19:33.628,01:28.783,98.69%,12,0,1900-01-01 00:01:28.783,40,2,2,False
2,lotus_98t,McLarenTim,13,+00:16.970,01:29.313,98.03%,0,0,1900-01-01 00:01:29.313,36,7,0,False
3,lotus_98t,NotJames,13,+00:24.942,01:30.070,98.64%,0,0,1900-01-01 00:01:30.070,32,6,0,False
4,lotus_98t,gunciz,13,+00:25.596,01:29.603,98.58%,0,0,1900-01-01 00:01:29.603,28,5,0,False
5,lotus_98t,Waffles,13,+00:34.353,01:29.055,95.69%,0,0,1900-01-01 00:01:29.055,25,1,3,False
6,lotus_98t,Salzeder,13,+00:43.224,01:30.079,97.21%,0,0,1900-01-01 00:01:30.079,22,4,0,False
7,lotus_98t,MrGrinder,13,+00:49.823,01:29.647,94.81%,0,0,1900-01-01 00:01:29.647,19,3,1,False
8,lotus_98t,Trempale,13,+01:34.906,01:30.240,95.22%,0,0,1900-01-01 00:01:30.240,16,8,0,False
9,lotus_98t,marzen224,12,+02:17.893,01:34.485,94.08%,0,0,1900-01-01 00:01:34.485,13,9,0,False


,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,DNF
Pos,,,,,,,,,,,,
1,lotus_98t,NotJames,13,19:44.906,01:29.968,99.41%,11,0,1900-01-01 00:01:29.968,40,3,False
2,lotus_98t,gunciz,13,+00:05.046,01:29.492,98.50%,0,0,1900-01-01 00:01:29.492,36,4,False
3,lotus_98t,Electro,13,+00:05.641,01:28.960,98.88%,0,0,1900-01-01 00:01:28.960,32,1,False
4,lotus_98t,McLarenTim,13,+00:09.548,01:28.835,97.20%,0,0,1900-01-01 00:01:28.835,28,2,False
5,lotus_98t,MrGrinder,13,+00:28.254,01:29.647,97.63%,0,0,1900-01-01 00:01:29.647,25,7,False
6,lotus_98t,Trempale,13,+00:39.579,01:30.105,97.67%,0,0,1900-01-01 00:01:30.105,22,8,False
7,lotus_98t,Salzeder,13,+00:42.971,01:30.983,97.64%,0,0,1900-01-01 00:01:30.983,19,6,False
8,lotus_98t,marzen224,12,+01:48.597,01:34.680,96.80%,0,0,1900-01-01 00:01:34.680,16,9,False
9,lotus_98t,"G, T",12,+02:27.599,01:34.833,90.67%,0,0,1900-01-01 00:01:34.833,13,10,False


Creating race report: F1H imola_5
no csv found for imola_5
Creating race report: F1H imola_6
no csv found for imola_6
Creating race report: F1H imola_7
no csv found for imola_7
Creating race report: F1H imola_8
no csv found for imola_8


### Construct Points Table

In [23]:
# compressed race result info obj for a driver and session

class DriverRaceResultInfo:
    
    def __init__(self, pos=-1, points=0, qualy_pos=-1, qualy_points=0, dnf=False):
        self.pos = pos
        self.points = points
        self.qualy_pos = qualy_pos
        self.qualy_points = qualy_points
        self.dnf = dnf
        self.participated = not bool(pos==-1 and qualy_pos==-1)
        self.total_points = self.points + self.qualy_points
        
    def __str__(self):
        if (self.pos<0) and (self.qualy_pos<0):
            return "NP"
        elif self.dnf:
            return "DNF"
        else:
            return str(self.total_points)
        
    def __repr__(self):
        return "DriverRaceResultInfo({},{},{},{},{})".format(self.pos, self.points, self.qualy_pos, self.qualy_points, self.dnf)
    

In [24]:
points_table = pd.DataFrame(index=series_drivers_table.index, columns=pd.MultiIndex.from_product([series_tracks_table.index, series_race_sessions], names=["track", "session"]))

# loop through drivers index
# construct series with the multindex
# go thorugh race reports and count up points

for driver in points_table.index:
    for track, session in points_table.columns:
        
        result_info = DriverRaceResultInfo()

        if track in race_reports:
            
            race_table = race_reports[track].tables[session]
            driver_race_result = race_table[race_table["Driver"]==driver]

            if len(driver_race_result==1):
                pos = driver_race_result.index[0]
                points = driver_race_result["Points"].item()
                qualify_pos = driver_race_result["Grid"].item()
                qualify_points = driver_race_result["Qualify Points"].item() if "Qualify Points" in driver_race_result.columns else 0
                dnf = driver_race_result["DNF"].item()
                
                result_info = DriverRaceResultInfo(pos, points, qualify_pos, qualify_points, dnf)
        
        points_table.loc[driver, (track, session)] = result_info
        
# truncate results to only calculate up to rounds needed

points_table = points_table.iloc[:, 0:(rounds_to_include*len(series_race_sessions))]

In [25]:
# keep seperate table with points totals and rounds participated

tracks = points_table.columns.unique(level="track")
points_totals_table = pd.DataFrame(index=series_drivers_table.index, columns=tracks)

def get_total_weekend_points(driver_session_results):
    return driver_session_results.apply(lambda result_info: result_info.total_points).sum()
for track in tracks:
    results_infos_for_weekend = points_table[track]
    points_totals_table[track] = results_infos_for_weekend.apply(get_total_weekend_points, axis=1)

points_totals_table["total"] = points_totals_table.agg(sum, axis=1)
points_totals_table["total_with_drop_week"] = points_totals_table.apply(lambda driver_row: driver_row["total"] - (min(driver_row)), axis=1)

def get_countback_str(driver_all_results):
    # convert driver results to an ascii string
    # a "higher" string value means more higher finishing positions
    countback_str = ""
    for pos in range(1, len(series_drivers_table)):
        num_finished_in_pos = driver_all_results.apply(lambda result_info: result_info.pos==pos).sum()
        countback_str += chr(48 + num_finished_in_pos)
    return countback_str
points_totals_table["countback_str"] = points_table.apply(get_countback_str, axis=1)

if drop_week:
    points_totals_table = points_totals_table.sort_values("countback_str", ascending=False)
    points_totals_table = points_totals_table.sort_values("total_with_drop_week", ascending=False, kind="mergesort") # mergesort is required to stable sort
else:
    points_totals_table = points_totals_table.sort_values("countback_str", ascending=False)
    points_totals_table = points_totals_table.sort_values("total", ascending=False, kind="mergesort")
    
points_table = points_table.reindex(points_totals_table.index)

display(points_table)
display(points_totals_table)

track                monza_1                     monza_2                \
session        Race 1 result Race 2 result Race 1 result Race 2 result   
ign                                                                      
Electro                   38            40            43            40   
McLarenTim                22            36            38            36   
gunciz                    28            25            33            28   
NotJames                  40            32            NP            NP   
MrGrinder                 32            19            28            32   
Waffles                   29            28            NP            NP   
Salzeder                  NP            NP            19            16   
Georgin                   19            22            25            25   
Viperion_NZ               13            NP            16            19   
Smartpipe                 16            16            22            22   
marzen224                 10            13            NP            NP   
Trempale                  NP            NP            NP            NP   
HonoredGoat               NP            NP            NP            NP   
G, T                      NP            NP            NP            NP   
Barricade                 NP            NP            NP            NP   
Horanjo                   NP            NP            NP            NP   
Monochromatica            NP            NP            NP            NP   
nemaides                  NP            NP            NP            NP   
mitski s                  NP            NP            NP            NP   
Birb                      NP            NP            NP            NP   

track                imola_3                     imola_4                
session        Race 1 result Race 2 result Race 1 result Race 2 result  
ign                                                                     
Electro                   43            36            42            32  
McLarenTim                36            40            36            28  
gunciz                    25            10            28            36  
NotJames                  34            32            32            40  
MrGrinder                 19            28            20            25  
Waffles                   29            25            28           DNF  
Salzeder                  13            22            22            19  
Georgin                   NP            NP            NP            NP  
Viperion_NZ               22            19            NP            NP  
Smartpipe                 NP            NP            NP            NP  
marzen224                 16             8            13            16  
Trempale                  NP            NP            16            22  
HonoredGoat               10            16            NP            NP  
G, T                      NP            NP            10            13  
Barricade                  8            13            NP            NP  
Horanjo                   NP            NP            NP            NP  
Monochromatica            NP            NP            NP            NP  
nemaides                  NP            NP            NP            NP  
mitski s                  NP            NP            NP            NP  
Birb                      NP            NP            NP            NP

track,monza_1,monza_2,imola_3,imola_4,total,total_with_drop_week,countback_str
ign,,,,,,,
Electro,78,83,79,74,314,240,5210000000000000000
McLarenTim,58,74,76,64,272,214,1501010000000000000
gunciz,53,61,35,64,213,178,0112300001000000000
NotJames,72,0,66,72,210,210,2040000000000000000
MrGrinder,51,60,47,45,203,158,0022103000000000000
Waffles,57,0,54,38,149,149,0003200001000000000
Salzeder,0,35,35,41,111,111,0000022110000000000
Georgin,41,50,0,0,91,91,0000211000000000000
Viperion_NZ,13,35,41,0,89,89,0000012110000000000


### Print Driver's Standings

##### Header

In [26]:
pos_width = 55
driver_width = 170
result_width = 45
track_width = len(series_race_sessions)*result_width
points_width = 92
table_width = pos_width + driver_width + len(series_tracks_table)*track_width + points_width

In [27]:
header_0_format = """(% border="1" cellpadding="1" style="width:{table_width}px"%)"""
header_1_pos_driver_format = """|=(% colspan="1" rowspan="2" scope="row" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {pos_width}px" %)Pos|=(% colspan="1" rowspan="2" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {driver_width}px;" %)Driver"""
header_1_track_format = """|=(% colspan="2" rowspan="1" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {track_width}px" %)((({header_1_track_flag_and_abbrev})))"""
header_1_track_flag_and_abbrev_format = """
[[image:{track_flag}||height="14" width="23"]]

{track_abbrev}
"""
header_1_points_format = """|=(% colspan="1" rowspan="2" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {points_width}px" %)Points"""
header_2_session_format = """|(% style="background-color:#eaecf0; text-align:center; vertical-align:middle; width:{result_width}px" %)**{session_abbrev}**"""

In [28]:
result_color_top_3 = ["#ffffbf", "#dfdfdf", "#ffdf9f"] 
result_color_points = "#dfffdf"
result_color_no_points = "#cfcfff"
result_color_ret = "#efcfff"
result_color_default = "#ffffff"

In [29]:
header_0 = header_0_format.format(table_width=table_width)

header_1_substrings = [header_1_pos_driver_format.format(pos_width=pos_width, driver_width=driver_width)]
for track in series_tracks_table.index:
    header_1_track_flag_and_abbrev = header_1_track_flag_and_abbrev_format.format(track_flag=series_tracks_table.loc[track]["flag"], track_abbrev=series_tracks_table.loc[track]["abbrev"])
    header_1_track = header_1_track_format.format(track_width=track_width, header_1_track_flag_and_abbrev=header_1_track_flag_and_abbrev)
    header_1_substrings.append(header_1_track)
header_1_substrings.append(header_1_points_format.format(points_width=points_width))
header_1 = "".join(header_1_substrings)

header_2_substrings = []
for _ in series_tracks_table.index:
    for session in series_race_sessions:
        session_number = re.findall('\d', session)[0]
        session_abbrev = "R{}".format(session_number)
        header_2_session = header_2_session_format.format(result_width=result_width, session_abbrev=session_abbrev)
        header_2_substrings.append(header_2_session)
header_2 = "".join(header_2_substrings)

lines_buffer = [header_0, header_1, header_2]

##### Driver Results

In [30]:
driver_row_pos_format = """|=(% style="text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width:{pos_width}px" %){pos}"""
driver_row_driver_format = """|(% style="width:{driver_width}px" %)[[image:{driver_flag}||height="14" width="23"]] {driver}"""
driver_row_result_format = """|(% style="background-color:{result_color}; text-align:center; vertical-align:middle; width:{result_width}px" %){result}"""
driver_row_points_format = """|(% style="text-align:center; vertical-align:middle; width:{points_width}px" %){points}"""

In [31]:
for pos, driver in enumerate(points_table.index):
    pos += 1
    
    driver_row_pos = driver_row_pos_format.format(pos_width=pos_width, pos=pos)
    
    driver_info = series_drivers_table.loc[driver]
    driver_flag = driver_info["flag"]
    driver_full_name = driver_info["name"]
    driver_row_driver = driver_row_driver_format.format(driver_width=driver_width, driver_flag=driver_flag, driver=driver_full_name)
    
    driver_row_substrings = [driver_row_pos, driver_row_driver]
    
    for i in range(num_total_races):
        
        result_string = ""
        result_color = result_color_default
        
        if i >= len(points_table.columns):
            driver_row_result = driver_row_result_format.format(result_color=result_color, result_width=result_width, result=result_string)
            driver_row_substrings.append(driver_row_result)
            continue
            
        result_info = points_table.loc[driver, points_table.columns[i]]
        
        if result_info.pos > 0:
            result_color = result_color_no_points
            result_string = str(result_info.pos)
            if result_info.points > 0:
                result_color = result_color_points
            if result_info.pos <= 3:
                result_color = result_color_top_3[result_info.pos-1]
            if result_info.dnf:
                result_string = "RET"
                result_color = result_color_ret
            if result_info.qualy_points > 0:
                result_string = "{}^^{}^^".format(result_string, result_info.qualy_pos)
        
        driver_row_result = driver_row_result_format.format(result_color=result_color, result_width=result_width, result=result_string)
        driver_row_substrings.append(driver_row_result)
        
    driver_totals = points_totals_table.loc[driver]
    driver_total = driver_totals["total"]
    
    if not points_table.loc[driver].apply(lambda r_info: r_info.participated).any():
        continue
    
    if drop_week:
        driver_total_with_drop_week = driver_totals["total_with_drop_week"]
        points_string = "**{}**".format(driver_total_with_drop_week)
        if not driver_total_with_drop_week == driver_total:
            points_string = "{}^^{}^^".format(points_string, driver_totals["total"])
    else:
        points_string = "**{}**".format(driver_total)
            
    driver_row_points = driver_row_points_format.format(points_width=points_width, points=points_string)
    driver_row_substrings.append(driver_row_points)
    
    driver_row = "".join(driver_row_substrings)
    lines_buffer.append(driver_row)

In [32]:
output_file = "{}/drivers_standings.txt".format(series)

with open(output_file, "w+") as fp:
    table_string = "\n".join(lines_buffer) + "\n\n"
    fp.write(table_string)
    print("wrote drivers standings to", output_file)

wrote drivers standings to F1H/drivers_standings.txt


### Debug

In [33]:
# test_series_directory = "MX5"
# test_race_directory = "donington"
# test_table_names = ["Qualify result", "Race 1 result", "Race 2 result"]

# testReport = RaceReport(test_table_names, test_series_directory, test_race_directory, csv_manual_adjustment=0, debug=True)
# testReport.write_generated_tables()

In [34]:
# table_width = pos_width + driver_width + 1*track_width + points_width

# print(header_0.format(table_width=table_width))
# print(header_1_pos_driver.format(pos_width=pos_width, driver_width=driver_width), end="")
# header_zolder_track_flag_and_abbrev = header_track_flag_and_abbrev.format(track_flag="https://upload.wikimedia.org/wikipedia/commons/thumb/9/92/Flag_of_Belgium_%28civil%29.svg/23px-Flag_of_Belgium_%28civil%29.svg.png", track_abbrev="ZOL")
# print(header_1_track.format(track_width=track_width, header_track_flag_and_abbrev=header_zolder_track_flag_and_abbrev), end="")
# print(header_1_points.format(points_width=points_width))

# for session in series_race_sessions:
#     session_number = re.findall('\d', session)[0]
#     session_abbrev = "R{}".format(session_number)
#     print(header_2_session.format(result_width=result_width, session_abbrev=session_abbrev), end="")
    
# print()

In [35]:
# TODO: ordering people tied on points based on countback
# TODO: manually fix driver renames
# TODO: do f1 history
# TODO: migrate to class based project, pycharm with main methods